### **Génération Dynamique d'un Contrat Commercial avec ReportLab et Platypus : Création et Gestion des Sections Dynamique pour un Contrat Personnalisé"**

#### *Description : Ce notebook guide la création d'un contrat commercial dynamique, où les sections du contrat sont générées à partir de données d'entrée. Utilisant les bibliothèques Python ReportLab et Platypus, chaque section du contrat est facilement modifiable et structurée, permettant une personnalisation complète des informations telles que les services, la compensation, et les informations des parties prenantes.*

In [1890]:
from docutils.utils.math.tex2unichar import space
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import A4
from reportlab.platypus import BaseDocTemplate, Frame, PageTemplate, Table, TableStyle, Paragraph, PageBreak, Image, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from datetime import datetime
import uuid
from faker import Faker
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

from io import BytesIO

# Génération du rapprot

## Étape 1 : Planifier la structure du document

### 1. **Analyse du contrat :**
   - Identification des grandes sections du document, comme :  
     - Titre  
     - Informations générales  
     - Sections clés (SERVICES, COMPENSATION, CONFIDENTIALITÉ, etc.).  
     - Signatures.  
   - Déterminez où chaque section doit apparaître et la structure visuelle (alignement, espaces, etc.).

### 2. **Découper les sections :**
   - Chaque section sera une unité distincte que vous allez intégrer dans votre document.
   - Posez-vous ces questions :
     - Quelles sections doivent être sur une nouvelle page ?
     - Quelles sections peuvent être regroupées ?
   - Créez une liste ou un schéma avec l’ordre d’apparition des sections.

#### a. INTRODUCTION

In [1891]:
INTRO_DATA_DICT_KEYS = ['company_name', 'company_status', 'company_jurisdiction',
                        'company_address', 'client_name', 'client_jurisdiction', 'client_address']

def intro(intro_data: dict, styles):
    if not intro_data:
        raise ValueError("intro_data is empty")
    # if not isinstance(styles, dict) or not all(isinstance(v, ParagraphStyle) for v in styles.values()):
    #     raise ValueError("styles must be a dictionary with ParagraphStyle objects")
    for key in INTRO_DATA_DICT_KEYS:
        if key not in intro_data.keys():
            raise ValueError(f"{key} not in intro data")
    
    today = datetime.today()
    formatted_date = today.strftime("%d/%m/%Y")
    
    # Construction des paragraphes avec les données dynamiques
    items = [
        Paragraph(
            f"Ce Contrat Commercial (« Contrat ») est conclu à ce <b>{formatted_date}</b> (« Date d'entrée en vigueur ») par et entre", 
            styles['IntroStyle']  # Utilisation de style approprié
        ),
        Paragraph(
            f" <b>{intro_data['company_name']}</b>, une <b>{intro_data['company_status']}</b> organisée selon les lois de "
            f"<b>{intro_data['company_jurisdiction']}</b> avec son principal lieu d'activité à "
            f"<b>{intro_data['company_address']}</b> (« Votre entreprise »),", 
            styles['IntroStyle']
        ),
        Paragraph("et", styles['IntroStyle']),
        Paragraph(
            f"<b>{intro_data['client_name']}</b>, une <b>{intro_data['client_status']}</b> organisée selon les lois de "
            f"<b>{intro_data['client_jurisdiction']}</b> avec son principal établissement à "
            f"<b>{intro_data['client_address']}</b> (« Client »)", 
            styles['IntroStyle']
        )
    ]
    
    return items


#### b. SERVICE

In [1892]:
def services_section(ser: str):
    if not ser:
        raise ValueError("services_data is empty")
    return f"""
        Votre Société s'engage à fournir les services suivants au Client : 
        <b>{ser}</b>.
        """


### c. TERME

In [1893]:
def term_section(term: str):
    if not term:
        raise ValueError("term is empty")
    return f"""
        La durée du présent contrat sera de <b>{term}</b>, 
        à compter de la date d'entrée en vigueur, à moins qu'il ne soit résilié plus tôt 
        conformément aux termes du présent contrat.
        """


#### d. COMPENSATION

In [1894]:
def compensation_section(compensation: str):
    if not compensation:
        raise ValueError("compensation is empty")
    return f"""
        Le Client s'engage à verser à Votre Société la rémunération suivante 
        pour les services rendus : <b>{compensation}</b>.
        """


#### e. CONFIDENTIALITÉ

In [1895]:
def confidentiality_section():
    return """
        Votre Société et le Client reconnaissent que pendant la durée du présent Contrat, 
        ils peuvent avoir accès aux informations confidentielles de l'autre partie. 
        Les deux parties conviennent de maintenir la confidentialité de ces informations 
        et de ne pas les divulguer à des tiers sans le consentement écrit préalable de l'autre partie, sauf si la loi l'exige.
        """


#### e. PROPRIÉTÉ INTELLECTUELLE

In [1896]:
def intellectual_property_section():
    return """
        Toute propriété intellectuelle créée par Votre Société dans le cadre de la fourniture 
        des services en vertu du présent contrat restera la propriété de Votre Société, sauf accord écrit contraire. 
        Le Client disposera d'une licence non exclusive et non transférable pour utiliser cette propriété intellectuelle 
        uniquement aux fins pour lesquelles les services sont fournis.
        """


#### f. TERMINAISON

In [1897]:
TERMINATION_DATA_DICT_KEYS = ['termination_notice', 'termination_remedy']

def termination_section(termination_data: dict):
    if not termination_data:
        raise ValueError("termination_data is empty")
    for key in TERMINATION_DATA_DICT_KEYS:
        if key not in termination_data.keys():
            raise ValueError(f"{key} not in termination data")
    return f"""
        Chacune des parties peut résilier le présent Contrat pour des raisons de commodité moyennant un préavis écrit de 
        <b>{termination_data['termination_notice']} jours</b> à l'autre partie. Chaque partie peut également résilier le présent Contrat 
        pour un motif valable en cas de manquement important de la part de l'autre partie, à condition qu'un avis écrit de ce manquement 
        soit donné et que la partie défaillante dispose de <b>{termination_data['termination_remedy']} jours</b> pour remédier au manquement 
        après avoir reçu cet avis.
        """


#### g. LOI APPLICABLE

In [1898]:
LAW_DATA_DICT_KEYS = ['applicable_law', 'arbitration_rules']

def law_section(law_content: dict):
    if not law_content:
        raise ValueError("law_data is empty")
    for key in LAW_DATA_DICT_KEYS:
        if key not in law_content.keys():
            raise ValueError(f"{key} not in law data")
    return f"""
        Ce Contrat sera régi et interprété conformément aux lois de <b>{law_content['applicable_law']}</b>. 
        Tout litige découlant de ou lié au présent Contrat sera résolu par arbitrage conformément aux règles de 
        <b>{law_content['arbitration_rules']}</b>, et le jugement sur la sentence rendue par le ou les arbitres pourra être 
        inscrit devant tout tribunal compétent.
        """


#### h. Limitation des responsabilités

In [1899]:
def limitation_responsabilite_section():
    return """
    En aucun cas, l'une ou l'autre des parties ne pourra être tenue responsable envers l'autre partie de tout dommage indirect, 
    accidentel, consécutif, spécial ou punitif découlant de ou lié au présent Contrat, même si la partie a été informée de la 
    possibilité de tels dommages.
    """

#### i. ACCORD INTÉGRAL

In [1900]:
def accord_integral_section():
    return """
                Ce Contrat contient l'intégralité de l'accord des parties en ce qui concerne l'objet des présentes et remplace tous les accords et ententes antérieurs et contemporains, qu'ils soient oraux ou écrits. Le présent Contrat ne peut être amendé ou modifié que par écrit signé par les deux parties.
            """

#### j. AFFECTATION

In [1901]:
def affectation_section():
    return """
            Aucune des parties ne peut céder ce Contrat ou tout droit ou obligation en vertu des présentes sans le consentement écrit préalable de l'autre partie, sauf que votre Société peut céder ce Contrat à toute société affiliée ou ayant cause sans un tel consentement.
        """

#### k. Divisibilité

In [1902]:
def divisibilite_section():
    return """
            Si une disposition de ce Contrat est jugée invalide, illégale ou inapplicable, les autres dispositions de ce Contrat ne seront pas affectées, et ce Contrat sera interprété comme si une telle disposition invalide, illégale ou inapplicable n'avait jamais été contenue dans les présentes. 
        """

#### l. SIGNATURES

In [1903]:
# Styles
signature_styles = {
    'DefaultStyle': ParagraphStyle(
        name='Default',
        fontName='Arial',
        fontSize=12,
        leading=15,
        spaceAfter=12
    ),
    'UnderlineStyle': ParagraphStyle(
        name='Underline',
        fontName='Arial',
        fontSize=12,
        leading=15,
        spaceAfter=12
    )
}

def generate_signatures_section(signatures_data: dict, signature_styles: dict, styles):
    """
    Génère les paragraphes pour la section des signatures.
    
    :param signatures_data: Dictionnaire contenant les données de l'entreprise et du client.
    :param signature_styles: Dictionnaire de ParagraphStyles pour formater le texte.
    :return: Liste de Paragraphs.
    """
    if not signatures_data:
        raise ValueError("signatures_data is empty")
    
    # Vérifier que les clés nécessaires sont présentes
    required_keys = ['company_name', 'company_email', 'company_address', 
                     'company_registration', 'client_name', 'client_email', 'client_address']
    for key in required_keys:
        if key not in signatures_data:
            raise ValueError(f"{key} is missing in signatures_data")

    # Liste des paragraphes à retourner
    items = []

    # Section Entreprise
    items.append(Paragraph("<b>ENTREPRISE</b>", styles['SubtitleStyle']))
    items.append(Paragraph(f"<u>{signatures_data['company_name']}</u>", signature_styles['UnderlineStyle']))
    items.append(Paragraph(f"Par: <u>_______________________</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Position: <u>_______________________</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Signature: <u>_______________________</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"E-mail: <u>{signatures_data['company_email']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Adresse: <u>{signatures_data['company_address']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Numéro d'enregistrement: <u>{signatures_data['company_registration']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(" ", signature_styles['DefaultStyle']))  # Espace visuel

    # Section Client
    items.append(Paragraph("<b>CLIENT</b>", styles['SubtitleStyle']))
    items.append(Paragraph(f"Nom du Client: <u>{signatures_data['client_name']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Signature: <u>_______________________</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Titre (le cas échéant): <u>_______________________</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"E-mail: <u>{signatures_data['client_email']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(f"Adresse: <u>{signatures_data['client_address']}</u>", signature_styles['DefaultStyle']))
    items.append(Paragraph(" ", signature_styles['DefaultStyle']))  # Espace visuel

    return items


## **Étape 2 : Configurer la page et les marges**

### 1. **Comprendre les dimensions de la page :**
   - Le format **A4** (595.27 x 841.89 points) est une bonne base pour un contrat.
   - Décidez des **marges externes** :
     - Exemples standards : 
       - Haut : 1 pouce (72 points).
       - Bas : 1 pouce.
       - Gauche/Droite : 0.75 pouce (54 points).

### 2. **Planifiez les marges internes (padding) pour le contenu :**
   - Déterminez combien d’espace laisser entre les bords du contenu et les bords internes de chaque section.
   - Décidez si certains éléments nécessitent plus de padding (par exemple, les sections importantes comme "CONFIDENTIALITÉ").


In [1904]:
def padding(left_padding:int|float=0.5, right_padding:int|float=0.5, top_padding:int|float=0.5, bottom_padding:int|float=0.5):
    """
    Si vous avez des blocs contenant des éléments textuels denses, comme des tableaux, utilisez un padding réduit :
    Exemple : leftPadding=18, rightPadding=18, topPadding=12, bottomPadding=12.
    Pour des zones importantes ou pour aérer la mise en page, utilisez un padding plus grand :
    Exemple : leftPadding=36, rightPadding=36, topPadding=36, bottomPadding=36.
    """
    
    
    return dict(leftPadding=left_padding*inch, rightPadding=right_padding*inch, topPadding=top_padding*inch, bottomPadding=bottom_padding*inch)

def margins(left_pargin:int|float=0.75, right_pargin:int|float=0.75, top_margin:int|float=1, bottom_margin:int|float=1):
    # Documents A4 classiques
    # 1 pouce en haut/bas, 0.75 pouce sur les côtés
    
    return dict(leftMargin=left_pargin*inch, rightMargin=right_pargin*inch, topMargin=top_margin*inch, bottomMargin=bottom_margin*inch)


### 3. **Créez un cadre (Frame) :**
   - Imaginez un cadre où tout votre contenu sera contenu. Définissez les marges et paddings pour ce cadre.


In [1905]:
def create_frame(padding_values: dict, a4_format: bool = False, **kwargs):
    """
    Crée un cadre (Frame) avec des valeurs de padding et des dimensions personnalisées ou A4.
    
    Arguments :
        padding_values (dict) : Dictionnaire contenant les paddings (left, right, top, bottom).
        a4_format (bool) : Utilise le format A4 si True.
        **kwargs : Dimensions personnalisées pour width et height si a4_format est False.

    Retourne :
        Frame : Objet Frame avec les dimensions et padding spécifiés.

    Exceptions :
        ValueError : Si width/height manquent ou padding_values n'est pas correctement formaté.
    """
    if not padding_values:
        raise ValueError("padding_values is missing or empty.")

    if a4_format:
        # Utilise le format A4
        return Frame(0, 0, *A4, **padding_values)
    else:
        # Dimensions personnalisées
        width = kwargs.get('width')
        height = kwargs.get('height')

        if width is None or height is None:
            raise ValueError("Height or width is missing.")

        if not isinstance(width, (float, int)) or not isinstance(height, (float, int)):
            raise ValueError("Width and height must be of type float or int.")

        return Frame(0, 0, width, height, **padding_values)


# **Étape 3 : Styliser le texte et les blocs**

## 1. **Décidez des styles pour chaque type de contenu :**

### a. **Titre principal :**
 - Taille de police (ex. : 18 ou 24 points).
 - Alignement (ex. : centré).
 - Espacement après (un grand espace pour démarquer). 

In [1906]:
# Enregistrer la police Arial
pdfmetrics.registerFont(TTFont('Arial', 'arial.ttf'))
pdfmetrics.registerFont(TTFont('Arial-Bold', 'arialbd.ttf'))  # Gras
pdfmetrics.registerFont(TTFont('Arial-Italic', 'ariali.ttf'))  # Italique
pdfmetrics.registerFont(TTFont('Arial-BoldItalic', 'arialbi.ttf'))  # Gras italique

In [1907]:
def get_title_style(
    font_name="Arial-Bold",
    font_size=18,
    font_color="black",
    alignment=0,  # 0 = left, 1 = center, 2 = right
    line_spacing=1.15,
    space_after=24,
    space_before=12,
    bold=True,
):
    return ParagraphStyle(
        name="TitleStyle",
        fontName=font_name,
        fontSize=font_size,
        textColor=font_color,
        alignment=alignment,
        leading=font_size * line_spacing,  # Interligne basé sur la taille de police
        spaceAfter=space_after,
        spaceBefore=space_before,  # Pas d'espace avant le titre
        fontWeight="bold" if bold else "normal",
    )


### b. **Sous-titres (ex. : SERVICES) :**
 - Taille moyenne (14 ou 16 points).
 - Police en gras ou avec un style particulier.

In [1908]:
def get_subtitle_style(
    font_name="Arial-Bold",
    font_size=14,
    font_color="black",
    alignment=0,  # Alignement à gauche
    line_spacing=1.15,
    space_after=12,
    space_before=12,
    bold=True,
):
    return ParagraphStyle(
        name="SubtitleStyle",
        fontName=font_name,
        fontSize=font_size,
        textColor=font_color,
        alignment=alignment,
        leading=font_size * line_spacing,
        spaceAfter=space_after,
        spaceBefore=space_before,
        fontWeight="bold" if bold else "normal",
    )


### c. **Texte normal (paragraphe) :**
 - Taille (12 points, standard pour les documents professionnels).
 - Police lisible (comme Times New Roman ou Helvetica). 

In [1909]:
def get_text_style(
    font_name="Arial",
    font_size=14,
    font_color="black",
    alignment=0,  # Alignement à gauche
    line_spacing=1.15,
    space_after=6,
    space_before=12,
    bold=False,
):
    return ParagraphStyle(
        name="TextStyle",
        fontName=font_name,
        fontSize=font_size,
        textColor=font_color,
        alignment=alignment,
        leading=line_spacing * font_size,
        spaceAfter=space_after,
        spaceBefore=space_before,
        fontWeight="bold" if bold else "normal",
    )


## 2. **Définissez les espacements :**
- Entre un titre et un paragraphe : spaceAfter=24 points pour donner un grand espacement.
- Entre les lignes de texte : Utilisation de l'interligne de 1.15x pour le texte normal (ajustable avec line_spacing).
- Espacement après les paragraphes : Par défaut, 6 points pour éviter que le texte suivant soit trop près.


# **Étape 4 : Ajouter les champs dynamiques**


# **Étape 5 : Ajouter les numéros de page**
1. **Décidez où afficher les numéros de page :**
   - En-tête ou pied de page ?
   - Alignez-les à droite, à gauche ou au centre ?
2. **Incluez un texte standard :**
   - Exemple : "Page X de Y".
3. **Réfléchissez à leur positionnement précis :**
   - Mesurez combien d’espace laisser entre le numéro de page et le bord de la page (ex. : 0.5 pouce ou 36 points).


In [1910]:
def page_footer(canvas, doc, pagesize=A4):
    page_num = canvas.getPageNumber()
    canvas.drawRightString(pagesize[0]/2, 20, "Page {}".format(page_num))


# **Étape 6 : Gérer les longues sections ou paragraphes**
1. **Coupez un paragraphe si nécessaire :**
   - Si un paragraphe est trop long, repérez une phrase ou une idée où vous pouvez le diviser naturellement.
   - Placez la première partie dans une section ou une colonne, et la deuxième partie dans une nouvelle section ou colonne.

2. **Vérifiez la lisibilité :**
   - Assurez-vous que le texte divisé reste cohérent et fluide pour le lecteur.


In [1911]:
def create_page_template(frames, a4_format=False, **kwargs):
    """
    Crée un PageTemplate avec des cadres (Frames) spécifiés et un pied de page.

    Arguments :
        frames (Frame ou list[Frame]) : Un ou plusieurs objets Frame à inclure dans le PageTemplate.
        a4_format (bool) : Si True, utilise le format A4 pour la taille de la page.
        **kwargs : Taille personnalisée de la page (pagesize=(width, height)).

    Retourne :
        PageTemplate : Un objet PageTemplate configuré.
    
    Exceptions :
        ValueError : Si frames ou pagesize est invalide.
    """
    # Générer un identifiant unique pour le modèle de page
    page_template_id = str(uuid.uuid4())  # Utilisation de uuid native

    # Valider les frames
    if isinstance(frames, (list, tuple)):
        for frame in frames:
            if not isinstance(frame, Frame):
                raise ValueError("All frames must be instances of Frame.")
    elif not isinstance(frames, Frame):
        raise ValueError("Frame must be of type Frame.")

    # Si le format A4 est demandé
    if a4_format:
        return PageTemplate(id=page_template_id, frames=frames, onPage=page_footer, pagesize=A4)
    
    # Vérifier la taille de page personnalisée
    page_size = kwargs.get('pagesize', None)
    if not isinstance(page_size, (tuple, list)) or len(page_size) != 2 or not all(isinstance(x, (float, int)) for x in page_size):
        raise ValueError("Page size must be a tuple or list of two numbers (width, height).")
    
    # Retourner le PageTemplate avec la taille personnalisée
    return PageTemplate(id=page_template_id, frames=frames, onPage=page_footer, pagesize=page_size)


In [1912]:
# Initialisation des styles
styles = getSampleStyleSheet()

# Titre principal
title_style = get_title_style(bold=True)
styles.add(title_style)

# Sous-titres
subtitle_style = get_subtitle_style(bold=True, space_before=32, space_after=32)
styles.add(subtitle_style)

# Texte normal (paragraphe)
text_style = get_text_style(space_after=0, space_before=0, line_spacing=1.5, font_size=14)
styles.add(text_style)

# Intro Style
intro_style =ParagraphStyle(
        name="IntroStyle",
        fontName="Arial",
        fontSize=14,
        textColor="black",
        alignment=0,
        leading=1.5 * 14,
        spaceAfter=6,
        spaceBefore=12,
        fontWeight="normal",
    )
styles.add(intro_style)

In [1913]:
elements = []
fake = Faker()
# Fonction générique pour ajouter une section
def add_section(elements, title, content, subtitle_style, text_style):
    """
    Ajoute une section au PDF.
    
    Arguments :
        elements (list) : Liste des éléments pour le PDF.
        title (str) : Titre de la section.
        content (str) : Contenu de la section.
        subtitle_style : Style pour le titre de la section.
        text_style : Style pour le contenu de la section.
    """
    elements.append(Paragraph(title, subtitle_style))
    # elements.append(Spacer(1, 12))  # Espace après le titre
    elements.append(Paragraph(content, text_style))
    # elements.append(Spacer(1, 24))  # Espace après le contenu

# **Titre Principal**
elements.append(Paragraph(
    "Contrat Commercial", 
    styles['TitleStyle']
))

# Ajouter une ligne sous le texte avec des tirets
elements.append(Paragraph("<u>_________________________________________________________</u>", styles['TextStyle']))

# Ajouter un espacement
elements.append(Spacer(1, 24))

# **Introduction**
intro_data = {
    'company_name': fake.company(),
    'company_status': "Société Anonyme (SA)",
    'company_jurisdiction': fake.country(),
    'company_address': fake.address(),
    'client_name': fake.company(),
    'client_status': "SA",
    'client_jurisdiction': fake.country(),
    'client_address': fake.address()
}

introduction = intro(intro_data, styles)
elements.extend(introduction)

# Ajouter une ligne sous le texte avec des tirets
elements.append(Paragraph("<u>_________________________________________________________</u>", styles['TextStyle']))

# Ajouter un espacement
elements.append(Spacer(1, 24))

# **Services**
services_data = "Assistance informatique, conseils stratégiques."
services = services_section(services_data)
add_section(elements, "SERVICES", services, styles['SubtitleStyle'], styles['TextStyle'])

# **Terme**
terme_data = " 1 an"
terme = term_section(terme_data)
add_section(elements, "TERME", terme, styles['SubtitleStyle'], styles['TextStyle'])

# **Compensation**
compensation_data = "une rémunération mensuelle de 5000 € pour les services fournis."
compensation = compensation_section(compensation_data)
add_section(elements, "COMPENSATION", compensation, styles['SubtitleStyle'], styles['TextStyle'])

# **Confidentialité**
confidentiality_data = confidentiality_section()
add_section(elements, "CONFIDENTIALITÉ", confidentiality_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Propriété Intellectuelle**
intellectual_property_data = intellectual_property_section()
add_section(elements, "PROPRIÉTÉ INTELLECTUELLE", intellectual_property_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Terminaison**
termination_section_data = {
    'termination_notice': 30,
    'termination_remedy': 15
}
termination_data = termination_section(termination_section_data)
add_section(elements, "TERMINAISON", termination_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Limitation de Responsabilité**
limitation_responsabilite_data = limitation_responsabilite_section()
add_section(elements, "LIMITATION DE RESPONSABILITÉ", limitation_responsabilite_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Loi Applicable**
law_data = {
    'applicable_law': "France",
    'arbitration_rules': "CCI (Chambre de Commerce Internationale)"
}
law_data_content = law_section(law_data)
add_section(elements, "LOI APPLICABLE", law_data_content, styles['SubtitleStyle'], styles['TextStyle'])

# **Accord Intégral**
accord_integral_data = accord_integral_section()
add_section(elements, "ACCORD INTÉGRAL", accord_integral_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Affectation**
affectation_data = affectation_section()
add_section(elements, "AFFECTATION", affectation_data, styles['SubtitleStyle'], styles['TextStyle'])

# **Divisibilité**
divisibilite_data = divisibilite_section()
add_section(elements, "DIVISIBILITÉ", divisibilite_data, styles['SubtitleStyle'], styles['TextStyle'])

# Signatures
# Données pour la section des signatures
signatures_data = {
    'company_name': 'XYZ Corp',
    'company_email': 'contact@xyzcorp.com',
    'company_address': '123 rue de Paris, 75001 Paris, France',
    'company_registration': 'FR123456789',
    'client_name': 'John Doe',
    'client_email': 'johndoe@example.com',
    'client_address': '456 avenue de Lyon, 69000 Lyon, France'
}

# Générer les paragraphes pour les signatures
signature_paragraphs = generate_signatures_section(signatures_data, signature_styles, styles)

# Ajouter les paragraphes au document
elements.extend(signature_paragraphs)



In [1914]:
padding_values = padding(left_padding=1, right_padding=1, top_padding=1, bottom_padding=1.3)
frames = create_frame(padding_values, True)
page_template = create_page_template(frames, True)
margins_values = margins(left_pargin=9.75, right_pargin=19.75)
base_doc = BaseDocTemplate("contrat_comercial.pdf", pageTemplates=[page_template])
base_doc.build(elements)